# Visualizing Milky Way Dwarf Companions in DigitalSky Dark Matter

We will start with a catalog of 53 nearby galaxies provided by Alex Drlica-Wagner and Brian Nord of the Dark Energy Survey. In the visualization we will highlight those galaxies that were discovered by the Dark Energy Survey. We will then generate the files needed to visualize this catalog in the Digital Sky Dark Matter software that runs on the Iziko Planetarium and Digital Dome.

In [1]:
# Astropy imports
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance
from astropy.table import Table, Column
from astropy.io import ascii
# Here we read the catalog which is in a .yaml file
import yaml
f = open('data/dwarfs.yaml')
# use safe_load instead load
dataMap = yaml.safe_load(f)
f.close()

In [9]:
#collapse the dictionary tree into something that can be converted into a simple table 
keys = ['name','ra', 'dec', 'distance','major_axis','refs']
flatDict=[]
for key in dataMap:
    keyDict = {x:dataMap[key][x] for x in keys}
    flatDict.append(keyDict)

dwarfTable=Table(flatDict)
dwarfTable

dec,distance,major_axis,name,ra,refs
float64,float64,float64,str24,float64,str25
36.18,30.0,3.9,Triangulum II,33.32,\citep{1503.05554}
17.05139,417.0,0.99,Leo T,143.7225,--
51.92,97.0,11.3,Ursa Major I,158.72,\citep{Simon:2007dq}
-52.28,95.5,0.42,Eridanus III,35.69,\citep{1503.02584}
-34.449,147.0,16.6,Fornax,39.997,\citep{Walker:2008ax}
26.8,47.0,nan,Bootes III,209.3,--
-50.966,105.0,8.2,Carina,100.4029,\citep{Walker:2008ax}
16.08194,23.0,4.4,Segue 1,151.76667,\citep{Simon:2010ek}
-50.16,120.0,2.23,Grus I,344.18,\citep{1503.02079}


In [11]:
# Here we will add an additional column which is 1 if that galaxy was discovered by DES, 0 otherwise. 
# This column will be used to color the galaxies in the final visualizaton
import numpy as np
byDES=np.zeros(len(dwarfTable))
for i in range(len(dwarfTable['refs'])):
  if ('1508.03622' in dwarfTable['refs'][i]):
    byDES[i]=1.
  elif ('1503.02584' in dwarfTable['refs'][i]):
    byDES[i]=1.
dwarfTable.add_column(Column(byDES,name='byDES'))  
dwarfTable.remove_column('refs')  
dwarfTable

dec,distance,major_axis,name,ra,byDES
float64,float64,float64,str24,float64,float64
36.18,30.0,3.9,Triangulum II,33.32,0.0
17.05139,417.0,0.99,Leo T,143.7225,0.0
51.92,97.0,11.3,Ursa Major I,158.72,0.0
-52.28,95.5,0.42,Eridanus III,35.69,1.0
-34.449,147.0,16.6,Fornax,39.997,0.0
26.8,47.0,nan,Bootes III,209.3,0.0
-50.966,105.0,8.2,Carina,100.4029,0.0
16.08194,23.0,4.4,Segue 1,151.76667,0.0
-50.16,120.0,2.23,Grus I,344.18,0.0


In [35]:
dwarfTable.show_in_browser(jsviewer=True)

### Write as VOTable
Digital Sky Dark Matter reads VOTables. Dark Matter can parse the UCDs, however, here we won't bother to set them. We will provide Dark Matter with a seperate xml file that tells it how to interpret the VOTable (which columns are coordinates, which are labels, which should be used to size or color the particles). We can now provide spherical coordinates (ra,dec,distance), no longer having to convert to cartesian.

In [13]:
# Write the VO Table
dwarfTable.write('data/MWdwarfs.xml',format='votable')

In [14]:
# Write the xml file that contains column descriptions
import xml.etree.cElementTree as ET

root = ET.Element("ParticleTable")
doc = ET.SubElement(root, "Columns")
ET.SubElement(doc, "AddField", data="ra", type="RA")
ET.SubElement(doc, "AddField", data="dec", type="DEC")
ET.SubElement(doc, "AddField", data="name", type="LABEL")
ET.SubElement(doc, "AddField", data="distance", type="DIST")
ET.SubElement(doc, "AddField", data="major_axis")
ET.SubElement(doc, "AddField", data="byDES")
tree = ET.ElementTree(root)
tree.write("data/MWdwarfs_fields.xml")

## Write a colormap file
Digital Sky Dark Matter reads colormaps in the partiview .cmap file. These can either be interpreted as continous or indexed colors. Here we will generate one with just two colors one for galaxies discovered by DES one for all others. 

In [25]:
cmapFile=open('data/DES.cmap', 'w')
cmapFile.write('2\n1 1 1 1\n1 0 0 1\n')
cmapFile.close()

# Tidal disruption of the Saggitarius Dwarf Galaxy
Here we will create the files necessary to visualize an N-body simulation of the tidal disruption of the Sagattarius Dwarf galaxy as described in Law and Majewski 2012 http://iopscience.iop.org/0004-637X/714/1/229/article Data files were taken from the website: http://www.astro.virginia.edu/~srm4n/Sgr/data.html
<img src="https://raw.githubusercontent.com/marksubbarao/pyWWT_AAS225/master/images/SgrStream.png">

In [21]:
#Import data set, keeping only the columns we need.
Stars = Table.read('data/SgrTriax_DYN.dat', format='ascii.basic')
Stars.keep_columns(["ra","dec","dist","Pcol"])
Stars

ra,dec,dist,Pcol
float64,float64,float64,int32
295.968,-42.1607,65.9751,8
284.571,-31.0457,28.7055,-1
300.004,-31.3875,24.1148,0
279.433,-35.746,62.4169,8
267.578,-38.2792,49.9106,7
176.893,1.10254,47.1228,4
104.444,33.1628,53.4371,4
71.5963,-0.34414,25.8991,5
201.532,39.4735,44.6561,8


In [26]:
# Write as VO Table
Stars.write('data/SGdwarf.xml',format='votable')

In [27]:
# Write the xml file that contains column descriptions
import xml.etree.cElementTree as ET

root = ET.Element("ParticleTable")
doc = ET.SubElement(root, "Columns")
ET.SubElement(doc, "AddField", data="ra", type="RA")
ET.SubElement(doc, "AddField", data="dec", type="DEC")
ET.SubElement(doc, "AddField", data="dist", type="DIST")
ET.SubElement(doc, "AddField", data="Pcol")
tree = ET.ElementTree(root)
tree.write("data/SGdwarf_fields.xml")

### Constructing th Colormap
Pcol indicates the pass when stars were stripped from the dwarf galaxy. We will constructing an identical colormap to the one used in the LM12 paper. Black is changed to grey for visability.

In [28]:
cmapFile=open('data/SG.cmap', 'w')
cmapFile.write('''2
0.5 0.5 0.5 1.0
1.0 0.5 0.0 1.0
1.0 0.5 0.0 1.0
1.0 0.0 1.0 1.0
1.0 0.0 1.0 1.0
0.0 1.0 1.0 1.0
0.0 1.0 1.0 1.0
0.0 1.0 0.1 1.0
''')
cmapFile.close()